In [15]:
import numpy as np
import pandas as pd

from ahc import AHC
from kmeans import K_Means

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
data = pd.read_csv("iris.csv")

X = data.iloc[:, :4]
# y = data.iloc[:, 4]

In [17]:
# k = int(input("Nro de clusters (k): "))
k = 3

In [18]:
def my_silhouette_score(X: np.ndarray, labels: np.array):
    # number of samples
    n = X.shape[0]

    # initialize silhouette scores
    sil_scores = np.zeros(n)

    # calculate pairwise distances manually
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = np.sqrt(np.sum((X[i] - X[j]) ** 2))

    for i in range(n):
        # Find the cluster of the current sample
        own_cluster = labels[i]

        # Mask for the own cluster
        mask_own = labels == own_cluster

        # a(i): the average distance to all other points in the same cluster
        if np.sum(mask_own) > 1:
            a_i = np.sum(D[i, mask_own]) / (np.sum(mask_own) - 1)
        else:
            a_i = 0

        # b(i): the minimum average distance to points in any other cluster
        b_i = np.inf
        for label in np.unique(labels):
            if label == own_cluster:
                continue
            mask_other = labels == label
            b_i = min(b_i, np.mean(D[i, mask_other]))

        # Compute the silhouette score for the current sample
        sil_scores[i] = (b_i - a_i) / max(a_i, b_i)

    return np.mean(sil_scores)

In [22]:
print("K-Means")
print("k: ", k)
model = K_Means(k=k)
labels, centroids = model.fit_predict(X.to_numpy())
model.print_centroids()
print("Labels: ")
print(labels)
print("Silhouette Score: ", my_silhouette_score(X.to_numpy(), labels))
# from sklearn.metrics import silhouette_score
# print("sklearn Silhouette Score:", silhouette_score(X.to_numpy(), labels))

K-Means
k:  3
Centroid 0: [6.92258065 2.89677419 5.48387097 1.83548387]
Centroid 1: [5.19875 3.1275  2.46375 0.6625 ]
Centroid 2: [6.30769231 3.02820513 5.04358974 1.79230769]
Labels: 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 1 2 1 1 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 0 2 0 2 0 0 2 0 0 0 2
 2 0 2 2 2 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 2 0 2 2 2 0 0 0 2 0 0 0 2 2
 2 2]
Silhouette Score:  0.48964165958580147


In [24]:
print("AHC")
ahc = AHC(X, save_output=False)
ahc.run()

AHC
